In [ ]:
import csv
import pandas as pd
from apple_store_scraper import AppStore
from concurrent.futures import ThreadPoolExecutor
from tenacity import retry, stop_never, wait_exponential, retry_if_exception_type
from requests.exceptions import RetryError

@retry(stop=stop_never, wait=wait_exponential(multiplier=1, max=600), retry=retry_if_exception_type(RetryError))
def get_reviews(app_name, country="cn"):
    app = AppStore(country=country, app_name=app_name)
    app.review(how_many=200000)
    return app.reviews

def save_reviews_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.to_csv(filename, mode='a', index=False)

def process_app(app_name):
    try:
        reviews = get_reviews(app_name)
        save_reviews_to_csv(reviews, f"{app_name}_reviews.csv")
    except Exception as e:
        print(f"Error processing app {app_name}: {e}")

def main():
    app_names = ["taobao"]  # Add your app names here
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(process_app, app_names)

if __name__ == "__main__":
    main()